In [7]:
%pip install pandas openpyxl
import pandas as pd

df = pd.DataFrame()
sheet = pd.read_excel('教育科學.xlsx', sheet_name=None, engine='openpyxl')
sheet_names = list(sheet.keys())
sheet_names.reverse()

for sheet_name in sheet_names[:]:
    sheet_df = sheet[sheet_name]
    df = pd.concat([df, sheet_df], ignore_index=True)

df.tail(20)

Note: you may need to restart the kernel to use updated packages.


,職類名稱,名次,提名單位
2046,國服,3,臺中市私立僑泰高級中學
2047,國服,4,國立關西高級中學
2048,國服,佳作,台南家專學校財團法人台南應用科技大學
2049,國服,佳作,高雄市立三民高級家事商業職業學校
2050,板金,1,國立成功大學附屬臺南工業高級中等學校
2051,板金,2,國立成功大學附屬臺南工業高級中等學校
2052,板金,3,新北市立三重高級商工職業學校
2053,板金,4,國立成功大學附屬臺南工業高級中等學校
2054,板金,5,國立新竹高級工業職業學校
2055,板金,佳作,國立新竹高級工業職業學校


In [8]:
# 獲取所有職類名稱和提名單位的唯一值
all_categories = df['職類名稱'].unique()
all_units = df['提名單位'].unique()

# 建立一個新的 DataFrame，包含所有職類名稱和提名單位的組合
all_combinations = pd.MultiIndex.from_product([all_categories, all_units], names=['職類名稱', '提名單位']).to_frame(index=False)

# 合併原始 DataFrame 和所有組合 DataFrame
merged_df = pd.merge(all_combinations, df, on=['職類名稱', '提名單位'], how='left')

# 將 NaN 值填充為 0
merged_df['名次'] = merged_df['名次'].fillna("缺失")

# 刪除重複的行，只保留第一個出現的
merged_df = merged_df.drop_duplicates(subset=['職類名稱', '提名單位'])


merged_df.head(20)

,職類名稱,提名單位,名次
0,綜合機械,臺中市立臺中工業高級中等學校,1
2,綜合機械,NaN,3
3,綜合機械,臺中市立大甲工業高級中等學校,4
4,綜合機械,國立羅東高級工業職業學校,5
6,綜合機械,國立新竹高級工業職業學校,佳作
7,綜合機械,國立屏東高級工業職業學校,缺失
8,綜合機械,臺北市立木柵高級工業職業學校,缺失
9,綜合機械,國立秀水高級工業職業學校,缺失
10,綜合機械,國立彰化師範大學附屬高級工業職業學校,缺失
11,綜合機械,國立員林高級農工職業學校,缺失


In [2]:
%pip install pandas scikit-learn openpyxl imbalanced-learn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

# 讀取 Excel 文件
excel_path = './教育科學.xlsx'
df = pd.read_excel(excel_path, engine='openpyxl')

# 資料預處理
# 將名次轉換為數值類別
df['名次'] = df['名次'].apply(lambda x: 0 if x == '佳作' else int(x))

# 將提名單位和職類名稱轉換為數值類別
label_encoder = LabelEncoder()
df['提名單位'] = label_encoder.fit_transform(df['提名單位'])
df['職類名稱'] = label_encoder.fit_transform(df['職類名稱'])

# 特徵選擇
X = df[['職類名稱']]
# y = df['名次']
y = df['提名單位']

# 分割資料集為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 處理資料不平衡
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# 訓練隨機森林模型
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_res, y_train_res)

# 預測
y_pred = clf.predict(X_test)

# 評估模型
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Note: you may need to restart the kernel to use updated packages.


c:\Users\adven\miniconda3\envs\deep-learning-d2l\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
c:\Users\adven\miniconda3\envs\deep-learning-d2l\lib\site-packages\sklearn\utils\_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


ValueError: Expected n_neighbors <= n_samples_fit, but n_neighbors = 6, n_samples_fit = 2, n_samples = 2